In [1]:
import ROOT, os, sys, glob
import numpy as np
import math
from matplotlib import pyplot as plt
from ROOT import kBlack, kBlue, kRed
from array import array

ROOT.EnableImplicitMT(4); 

Welcome to JupyROOT 6.28/00


In [2]:
list(glob.glob("/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/*125X_v0.2.root"))

['/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/l1MuNano_BsToMuMu_PU200.125X_v0.2.root',
 '/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/l1MuNano_DYToLL_M-10To50_PU200.125X_v0.2.root',
 '/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/l1MuNano_DYToLL_M-1To10_TuneCP5_14TeV-pythia8_PU200.125X_v0.2.root',
 '/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/l1MuNano_DYToLL_M-50_PU200.125X_v0.2.root',
 '/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/l1MuNano_HAHM_ZdToMuMu_M10_PU200.125X_v0.2.root',
 '/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/l1MuNano_HAHM_ZdToMuMu_M12_PU200.125X_v0.2.root',
 '/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/l1MuNano_HAHM_ZdToMuMu_M15_PU200.125X_v0.2.root',
 '/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/l1MuNano_HAHM_ZdToMuMu_M25_PU200.125X_v0.2.root',
 '/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/l1MuNano_HAHM_ZdToMuMu_M2_PU200.125X_v0.2.root',
 '/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout

In [3]:
def Normal_h(h1,h2):
    h_p = h1.Clone()
    if h_p.Integral(0,h_p.GetNbinsX()+1) > 0:
        h_p.Scale(h2.Integral(0,h_p.GetNbinsX()+1)/h_p.Integral(0,h_p.GetNbinsX()+1))
    return h_p;

In [4]:
def Efficiency_plotter(file_name,M):
    m =M
    #m = float(M.replace("p","."))
    rdf = ROOT.RDataFrame("Events",f"/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/{file_name}")
    def makeDimuonPairs(name, rdf):
        ## define 4-vectors in cartesian coordinates, for easier 
        rdf = rdf.Define(f"{name}Mu_p4", f"ROOT::VecOps::Construct<ROOT::Math::XYZTVector>(ROOT::VecOps::Construct<ROOT::Math::PtEtaPhiMVector>({name}Mu_pt,{name}Mu_eta,{name}Mu_phi,{name}Mu_mass))")
        ## make indices of all the pairs
        rdf = rdf.Define(f"{name}DiMu_indices", f"""
            ROOT::VecOps::RVec<std::pair<int,int>> ret; 
            for (int i = 0, n = n{name}Mu; i < n; ++i) 
                for (int i2 = i+1; i2 < n; ++i2) 
                    ret.emplace_back(i,i2);
            return ret;""")

        #Let us impose a selection criteria here


        ## Make the number of dimuons, for convenience later
        rdf = rdf.Define(f"n{name}DiMu", f"{name}DiMu_indices.size()")
        ## compute the mass, charge
        rdf = rdf.Define(f"{name}DiMu_mass", f"""
            ROOT::RVecF ret; 
            for (const auto & pair : {name}DiMu_indices) 
                ret.emplace_back(({name}Mu_p4[pair.first]+{name}Mu_p4[pair.second]).M());
            return ret;""")


        rdf = rdf.Define(f"{name}DiMu_charge", f"""
            ROOT::RVecI ret; 
            for (const auto & pair : {name}DiMu_indices) 
                ret.emplace_back({name}Mu_charge[pair.first]+{name}Mu_charge[pair.second]);
            return ret;""")



        rdf = rdf.Define(f"{name}DiMu_minIdx", f"""
            ROOT::RVecI ret;
            for (const auto & pair : {name}DiMu_indices) 
                ret.emplace_back(std::min(TkMu_GenMuIdx[pair.first],TkMu_GenMuIdx[pair.second]));
            return ret;""")


        rdf = rdf.Define(f"{name}DiMu_eta", f"""
            ROOT::RVecF ret;
            for (const auto & pair : {name}DiMu_indices) 
                ret.emplace_back(std::max(abs({name}Mu_eta[pair.first]),abs({name}Mu_eta[pair.second])));
            return ret;""")
        
        rdf=rdf.Define(f"{name}DiMu_dz",f"""
            ROOT::RVecF ret;
            for(const auto & pair : {name}DiMu_indices)
                ret.emplace_back(({name}Mu_vz[pair.first]-{name}Mu_vz[pair.second]));
            return ret;""")
        #min pT of thw two
        rdf = rdf.Define(f"{name}Mu_minPT",f"""
            ROOT::RVecF ret;
            for(const auto & pair : {name}DiMu_indices)
                ret.emplace_back(std::min({name}Mu_pt[pair.first],{name}Mu_pt[pair.second]));
            return ret;""") #

        return rdf;
    
    entries_no_pair = rdf.Count().GetValue()

    
####################################################################################################################################################################    
    def makeTk_DiMuonPairs(rdf):
    
        #this is to defne DiMuon pairs for different properties
        rdf = rdf.Define(f"TkDiMu_hwIsoSum", f"""
            ROOT::RVecI ret; 
            for (const auto & pair : TkDiMu_indices) 
                ret.emplace_back(0.03125*(std::max(TkMu_hwIsoSum[pair.first],TkMu_hwIsoSum[pair.second])));
            return ret;""")

        rdf = rdf.Define(f"TkDiMu_hwIsoSumAp", f"""
            ROOT::RVecI ret; 
            for (const auto & pair : TkDiMu_indices) 
                ret.emplace_back(0.03125*(std::max(TkMu_hwIsoSumAp[pair.first],TkMu_hwIsoSumAp[pair.second])));
            return ret;""")

        rdf = rdf.Define(f"TkDiMu_hwQual", f"""
            ROOT::RVecI ret; 
            for (const auto & pair : TkDiMu_indices) 
                ret.emplace_back(std::min(TkMu_hwQual[pair.first],TkMu_hwQual[pair.second]));
            return ret;""")

        return rdf;
    rdf_Tkdimu_one = makeDimuonPairs("Tk",rdf) 
    rdf_Tkdimu = makeTk_DiMuonPairs(rdf_Tkdimu_one)
    
    #h_TkDiMu_mass = rdf_Tkdimu.Histo1D(("","",50,0,50),"TkDiMu_mass")
    
    hNTkPairs = rdf_Tkdimu.Histo1D(("nTkDiMu","N(#mu#mu)_{Tk};Events",12,-2,10), "nTkDiMu")
    h_TkDiMu_mass = rdf_Tkdimu.Histo1D(("TkMass","m(#mu#mu)_{Tk};m (GeV);Pairs",50,0,50), "TkDiMu_mass")
    hTkPairCharge = rdf_Tkdimu.Histo1D(("TkCharge","q(#mu#mu)_{Tk};q(#mu_{1}) + q(#mu_{2});Pairs",8,-4,4), "TkDiMu_charge")
    hTkPair_Dz = rdf_Tkdimu.Histo1D(("TkPair_dz","dz(#mu#mu)_{Tk};z(#mu_{1}) - z(#mu_{2}cm",100,-10,10),"TkDiMu_dz")
    hTkPair_minPt = rdf_Tkdimu.Histo1D(("TkPair_minPt","min p_{T}(#mu#mu)_{Tk};pT GeV",100,0,10),"TkMu_minPT")
    hTkPair_Dz = rdf_Tkdimu.Histo1D(("TkPair_dz","dz(#mu#mu)_{Gen};z(#mu_{1}) - z(#mu_{2})cm",50,-10,10),"TkDiMu_dz")
    hTkPair_minPt = rdf_Tkdimu.Histo1D(("TkPair_minPt","min p_{T}(#mu#mu)_{Gen};pT GeV",50,0,10),"TkMu_minPT")
    hTkPair_hwIsoSum = rdf_Tkdimu.Histo1D(("Tk_IsoSum","Iso Sum(#mu#mu)_{Tk};;Pairs",20,0,20),"TkDiMu_hwIsoSum")
    hTkPair_hwIsoSumAp = rdf_Tkdimu.Histo1D(("Tk_IsoSumAp","Iso Sum Ap(#mu#mu)_{Tk};;Pairs",20,0,20),"TkDiMu_hwIsoSumAp")
    hTkPair_hwQual = rdf_Tkdimu.Histo1D(("Tk_Qual","Qual(#mu#mu)_{Tk};;Pairs",50,0,200),"TkDiMu_hwQual")
    hTkPair_hwQual = rdf_Tkdimu.Histo1D(("TkDiMu_eta","#Eta(#mu#mu)_{Tk};;Pairs",50,0,200),"TkDiMu_eta")

 
    entries_Tk_pair = rdf_Tkdimu.Count().GetValue()
    
#     c1 = ROOT.TCanvas("c1","c1", 2000, 800)
#     c1.Divide(4,2)
#     for i, h1 in enumerate([hNTkPairs, h_TkDiMu_mass, hTkPairCharge,hTkPair_Dz,hTkPair_minPt, hTkPair_hwQual,hTkPair_hwIsoSum, hTkPair_hwIsoSumAp]):
#         c1.cd(i+1)
#         h1.Draw()
#     c1.Draw()
#     c1.SaveAs(f"{m}_muon_pairs.png")

####################################################################################################################################################################
    
    rdf_genDiMu = makeDimuonPairs("Gen",rdf_Tkdimu)
    
    entries_gen_pair = rdf_genDiMu.Count().GetValue()
    
####################################################################################################################################################################    
    
    def indexSelect(rdf):
        rdf = rdf.Define("TkDiMu_indMatch","(TkDiMu_minIdx!=-1)" )
        rdf = rdf.Define(f"IdxDiMu_indices", f"TkDiMu_indices[TkDiMu_indMatch]")
        #rdf = rdf.Filter(f"Sum(TkDiMu_indMatch)  >= 1")
        rdf = rdf.Define(f"nTkDiMu_Idx",f"IdxDiMu_indices.size()")
        rdf = rdf.Define(f"TkDiMu_IdxMass",f"TkDiMu_mass[TkDiMu_indMatch]")
        rdf = rdf.Define(f"TkDiMu_IdxCharge",f"TkDiMu_charge[TkDiMu_indMatch]")
        rdf = rdf.Define(f"TkDiMu_Idxdz",f"TkDiMu_dz[TkDiMu_indMatch]")
        rdf = rdf.Define(f"TkDiMu_IdxMinPt",f"TkMu_minPT[TkDiMu_indMatch]")
        rdf = rdf.Define(f"TkDiMu_IdxhwIsoSum",f"TkDiMu_hwIsoSum[TkDiMu_indMatch]")
        rdf = rdf.Define(f"TkDiMu_IdxhwIsoSumAp",f"TkDiMu_hwIsoSumAp[TkDiMu_indMatch]")
        rdf = rdf.Define(f"TkDiMu_IdxhwQual",f"TkDiMu_hwQual[TkDiMu_indMatch]")
        rdf = rdf.Define("TkDiMu_IdxEta","TkDiMu_eta[TkDiMu_indMatch]")
        return rdf;

    rdf_idxSel = indexSelect(rdf_genDiMu)
    #hIdxTkPairMass = rdf_idxSel.Histo1D(("Tk_Mass_Idx","Idx m(#mu#mu)_{Tk};m (GeV);Pairs",50,0,50), "TkDiMu_IdxMass")
    
    hNIdxTkPairs = rdf_idxSel.Histo1D(("nTk_DiMu_Idx","Idx N(#mu#mu)_{Tk};Events",10,0,10), "nTkDiMu_Idx")
    hIdxTkPairMass = rdf_idxSel.Histo1D(("Tk_Mass_Idx","Idx m(#mu#mu)_{Tk};m (GeV);Pairs",50,0,50), "TkDiMu_IdxMass")
    hIdxTkPairCharge = rdf_idxSel.Histo1D(("Tk_Charge_Idx","Idx q(#mu#mu)_{Tk};q(#mu_{1}) + q(#mu_{2});Pairs",8,-4,4), "TkDiMu_IdxCharge")
    hIdxTkPairDz = rdf_idxSel.Histo1D((" Tk dz_Idx","Idx dz(#mu#mu)_{TK};z(#mu_{1}) - z(#mu_{2});Pairs",50,-10,10), "TkDiMu_Idxdz")
    hIdxTkPairMinPT = rdf_idxSel.Histo1D((" Tk minPt_Idx","Idx min pT(#mu#mu)_{Tk};pT(GeV));Pairs",50,0,10), "TkDiMu_IdxMinPt")
    hIdxTkPair_hwQual = rdf_idxSel.Histo1D((" Tk hwQual_Idx","Idx hwQual(#mu#mu)_{Tk};pT(GeV));Pairs",50,0,200), "TkDiMu_IdxhwQual")
    hIdxTkPair_hwIsoSum = rdf_idxSel.Histo1D((" Tk hwIsoSum_Idx","Idx hwIsoSum(#mu#mu)_{Tk};pT(GeV));Pairs",20,0,20), "TkDiMu_IdxhwIsoSum")
    hIdxTkPair_hwIsoSumAp = rdf_idxSel.Histo1D((" Tk hwIsoSum_Idx Ap","Idx hwIsoSum Ap(#mu#mu)_{Tk};pT(GeV));Pairs",20,0,20), "TkDiMu_IdxhwIsoSumAp")
    hIdxTkPair_Eta = rdf_idxSel.Histo1D(("TkDiMu Idx Eta","Di Mu IdxEta(#mu#mu)_{Tk};pT(GeV));Pairs",20,0,20), "TkDiMu_IdxEta")


    entries_idx_pair = rdf_idxSel.Count().GetValue()
    
#     c2 = ROOT.TCanvas("c2","c2", 2000, 800)
#     c2.Divide(4,2)
#     for j, h2 in enumerate([hNIdxTkPairs, hIdxTkPairMass, hIdxTkPairCharge, hIdxTkPairDz, hIdxTkPairMinPT, hIdxTkPair_hwQual, hIdxTkPair_hwIsoSum, hIdxTkPair_hwIsoSumAp]):
#         c2.cd(j+1)
#         h2.Draw()
#     c2.Draw()
#     c2.SaveAs(f"{m}_Index_matched_pairs.png")
    
####################################################################################################################################################################
    
    #hIdxTkPairMass = rdf_idxSel.Histo1D(("Tk_Mass_Idx","Idx m(#mu#mu)_{Tk};m (GeV);Pairs",50,0,50), "TkDiMu_IdxMass")

    def select_ind(name, rdf):
    
        #Let us impose a selection criteria here
        #rdf = rdf.Define(f"{name}DiMu_selind",f"{name}DiMu_indices")
        rdf = rdf.Define(f"{name}DiMu_good", f"({name}DiMu_charge<0.1 && {name}DiMu_charge>-0.1) &&({name}DiMu_dz<1 && ({name}DiMu_dz>-1))&&({name}Mu_minPT > 0.25*{name}DiMu_mass)&&({name}DiMu_eta <= 2) && (TkDiMu_hwIsoSumAp<=3) && (TkDiMu_hwIsoSum<=3)")                    
        rdf = rdf.Define(f"Good{name}DiMu_indices", f"{name}DiMu_indices[{name}DiMu_good]")
        rdf = rdf.Filter(f"Sum({name}DiMu_good)  >= 1")
        rdf = rdf.Define(f"n{name}DiMu_sel",f"Good{name}DiMu_indices.size()")
        rdf = rdf.Define(f"{name}DiMu_selMass",f"{name}DiMu_mass[{name}DiMu_good]")
        rdf = rdf.Define(f"{name}DiMu_selCharge",f"{name}DiMu_charge[{name}DiMu_good]")
        rdf = rdf.Define(f"{name}DiMu_seldz",f"{name}DiMu_dz[{name}DiMu_good]")
        rdf = rdf.Define(f"{name}DiMu_selMinPt",f"{name}Mu_minPT[{name}DiMu_good]")
        rdf = rdf.Define(f"{name}DiMu_selEta",f"{name}DiMu_eta[{name}DiMu_good]")

        #rdf = rdf.Define(f"GoodTkDiMu_minIdx_{name}",f"TkDiMu_minIdx[{name}DiMu_good]")
        return rdf;

    rdf_Tk_selFeat = select_ind("Tk",rdf_idxSel)
    #rdf_selFeat = select_ind("Gen", rdf_Tk_selFeat)
    
    h_TkDiMu_selMass = rdf_Tk_selFeat.Histo1D(("","",50,0,50),"TkDiMu_selMass")

    
    def select_idx_tk(rdf):

        #Let us impose a selection criteria here
        #rdf = rdf.Define(f"{name}DiMu_selind",f"{name}DiMu_indices")
        rdf = rdf.Define(f"TkDiMu_Idx_good", f"(TkDiMu_IdxCharge <0.1 && TkDiMu_IdxCharge > -0.1) &&(TkDiMu_Idxdz <1 && (TkDiMu_Idxdz > -1))&&(TkDiMu_IdxMinPt > 0.25*TkDiMu_IdxMass) && (TkDiMu_IdxhwQual>75) && (TkDiMu_IdxEta <=2)&& (TkDiMu_IdxhwIsoSum<=3) && (TkDiMu_IdxhwIsoSumAp<=3) ")                    
        rdf = rdf.Define(f"TkDiMu_sel_Idx_indices", f"IdxDiMu_indices[TkDiMu_Idx_good]")
        rdf = rdf.Filter(f"Sum(TkDiMu_Idx_good)  >= 1")
        rdf = rdf.Define(f"nTkDiMu_IdxSel",f"TkDiMu_sel_Idx_indices.size()")
        rdf = rdf.Define(f"TkDiMu_IdxselMass",f"TkDiMu_IdxMass[TkDiMu_Idx_good]")
        rdf = rdf.Define(f"TkDiMu_IdxselCharge",f"TkDiMu_IdxCharge[TkDiMu_Idx_good]")
        rdf = rdf.Define(f"TkDiMu_Idxseldz",f"TkDiMu_Idxdz[TkDiMu_Idx_good]")
        rdf = rdf.Define(f"TkDiMu_IdxselMinPt",f"TkDiMu_IdxMinPt[TkDiMu_Idx_good]")
        rdf = rdf.Define(f"TkDiMu_IdxselhwQual",f"TkDiMu_IdxhwQual[TkDiMu_Idx_good]")
        rdf = rdf.Define(f"TkDiMu_IdxselhwIsoSum",f"TkDiMu_IdxhwIsoSum[TkDiMu_Idx_good]")
        rdf = rdf.Define(f"TkDiMu_IdxselhwIsoSumAp",f"TkDiMu_IdxhwIsoSumAp[TkDiMu_Idx_good]")
        rdf = rdf.Define(f"GoodTkDiMu_minIdx",f"TkDiMu_minIdx[TkDiMu_Idx_good]")
        rdf = rdf.Define(f"TkDiMu_IdxselEta",f"TkDiMu_IdxEta[TkDiMu_Idx_good]")

        return rdf;
    
    rdf_Tk_IdxselFeat = select_idx_tk(rdf_Tk_selFeat)

    hNGoodIdxTkPairs = rdf_Tk_IdxselFeat.Histo1D(("good nTk_DiMu_sel","Good N(#mu#mu)_{Tk};Events",10,0,10), "nTkDiMu_IdxSel")
    hGoodIdxTkPairMass = rdf_Tk_IdxselFeat.Histo1D(("Good Tk_Mass","Good m(#mu#mu)_{Tk};m (GeV);Pairs",50,0,50), "TkDiMu_IdxselMass")
    hGoodIdxTkPairCharge = rdf_Tk_IdxselFeat.Histo1D(("Good Tk_Charge","Good q(#mu#mu)_{Tk};q(#mu_{1}) + q(#mu_{2});Pairs",8,-4,4), "TkDiMu_IdxselCharge")
    hGoodIdxTkPairDz = rdf_Tk_IdxselFeat.Histo1D(("Good Tk dz","Good dz(#mu#mu)_{TK};z(#mu_{1}) - z(#mu_{2});Pairs",50,-10,10), "TkDiMu_Idxseldz")
    hGoodIdxTkPairMinPT = rdf_Tk_IdxselFeat.Histo1D(("Good Tk minPt","Good min pT(#mu#mu)_{Tk};pT(GeV));Pairs",50,0,20), "TkDiMu_IdxselMinPt")
    hGoodIdxTkPairhwQual = rdf_Tk_IdxselFeat.Histo1D(("Good Tk hw Qual","Good hw Qual(#mu#mu)_{Tk};);Pairs",50,0,200), "TkDiMu_IdxselhwQual")
    hGoodIdxTkPairhwIsoSum = rdf_Tk_IdxselFeat.Histo1D(("Good Tk hw Iso Sum","Good hw Iso Sum(#mu#mu)_{Tk};);Pairs",20,0,20), "TkDiMu_IdxselhwIsoSum")
    hGoodIdxTkPairhwIsoSumAp = rdf_Tk_IdxselFeat.Histo1D(("Good Tk hw Iso Sum Ap","Good hw Iso Sum Ap(#mu#mu)_{Tk};);Pairs",20,0,20), "TkDiMu_IdxselhwIsoSumAp")
    hGoodIdxTkPairEta = rdf_Tk_IdxselFeat.Histo1D(("Good TkDiMu Eta","Good TkDiMu lEta(#mu#mu)_{Tk};);Pairs",20,0,20), "TkDiMu_IdxselEta")

    entries_selIdx_pair = rdf_Tk_IdxselFeat.Count().GetValue()
    

    
    return dict(entries_no_pair = entries_no_pair, hNTkPairs=hNTkPairs, h_TkDiMu_mass=h_TkDiMu_mass, hTkPairCharge=hTkPairCharge, hTkPair_Dz=hTkPair_Dz,hTkPair_minPt=hTkPair_minPt,hTkPair_hwIsoSum=hTkPair_hwIsoSum, hTkPair_hwIsoSumAp=hTkPair_hwIsoSumAp, hTkPair_hwQual=hTkPair_hwQual, hNIdxTkPairs = hNIdxTkPairs, hIdxTkPairMass = hIdxTkPairMass, hIdxTkPairCharge =  hIdxTkPairCharge, hIdxTkPairDz = hIdxTkPairDz, hIdxTkPairMinPT = hIdxTkPairMinPT, hIdxTkPair_hwQual = hIdxTkPair_hwQual, hIdxTkPair_hwIsoSum = hIdxTkPair_hwIsoSum, hIdxTkPair_hwIsoSumAp = hIdxTkPair_hwIsoSumAp ,  hNGoodIdxTkPairs = hNGoodIdxTkPairs, hGoodIdxTkPairMass = hGoodIdxTkPairMass, hGoodIdxTkPairCharge = hGoodIdxTkPairCharge, hGoodIdxTkPairDz = hGoodIdxTkPairDz, hGoodIdxTkPairMinPT = hGoodIdxTkPairMinPT, hGoodIdxTkPairhwQual = hGoodIdxTkPairhwQual, hGoodIdxTkPairhwIsoSum = hGoodIdxTkPairhwIsoSum, hGoodIdxTkPairhwIsoSumAp = hGoodIdxTkPairhwIsoSumAp, hGoodIdxTkPairEta = hGoodIdxTkPairEta);






In [5]:
def Efficiency_SN(file_name,M):
    
    #m = float(M.replace("p","."))
    m = M
    rdf = ROOT.RDataFrame("Events",f"/eos/cms/store/cmst3/group/l1tr/gpetrucc/dilep-scout/{file_name}")
    def makeDimuonPairs(name, rdf):
        ## define 4-vectors in cartesian coordinates, for easier 
        rdf = rdf.Define(f"{name}Mu_p4", f"ROOT::VecOps::Construct<ROOT::Math::XYZTVector>(ROOT::VecOps::Construct<ROOT::Math::PtEtaPhiMVector>({name}Mu_pt,{name}Mu_eta,{name}Mu_phi,{name}Mu_mass))")
        ## make indices of all the pairs
        rdf = rdf.Define(f"{name}DiMu_indices", f"""
            ROOT::VecOps::RVec<std::pair<int,int>> ret; 
            for (int i = 0, n = n{name}Mu; i < n; ++i) 
                for (int i2 = i+1; i2 < n; ++i2) 
                    ret.emplace_back(i,i2);
            return ret;""")

        #Let us impose a selection criteria here


        ## Make the number of dimuons, for convenience later
        rdf = rdf.Define(f"n{name}DiMu", f"{name}DiMu_indices.size()")
        ## compute the mass, charge
        rdf = rdf.Define(f"{name}DiMu_mass", f"""
            ROOT::RVecF ret; 
            for (const auto & pair : {name}DiMu_indices) 
                ret.emplace_back(({name}Mu_p4[pair.first]+{name}Mu_p4[pair.second]).M());
            return ret;""")


        rdf = rdf.Define(f"{name}DiMu_charge", f"""
            ROOT::RVecI ret; 
            for (const auto & pair : {name}DiMu_indices) 
                ret.emplace_back({name}Mu_charge[pair.first]+{name}Mu_charge[pair.second]);
            return ret;""")



        rdf = rdf.Define(f"{name}DiMu_minIdx", f"""
            ROOT::RVecI ret;
            for (const auto & pair : {name}DiMu_indices) 
                ret.emplace_back(std::min(TkMu_GenMuIdx[pair.first],TkMu_GenMuIdx[pair.second]));
            return ret;""")


        rdf = rdf.Define(f"{name}DiMu_eta", f"""
            ROOT::RVecF ret;
            for (const auto & pair : {name}DiMu_indices) 
                ret.emplace_back(std::max(abs({name}Mu_eta[pair.first]),abs({name}Mu_eta[pair.second])));
            return ret;""")
        
        rdf=rdf.Define(f"{name}DiMu_dz",f"""
            ROOT::RVecF ret;
            for(const auto & pair : {name}DiMu_indices)
                ret.emplace_back(({name}Mu_vz[pair.first]-{name}Mu_vz[pair.second]));
            return ret;""")
        #min pT of thw two
        rdf = rdf.Define(f"{name}Mu_minPT",f"""
            ROOT::RVecF ret;
            for(const auto & pair : {name}DiMu_indices)
                ret.emplace_back(std::min({name}Mu_pt[pair.first],{name}Mu_pt[pair.second]));
            return ret;""") #

        return rdf;
    
    entries_no_pair = rdf.Count().GetValue()

    
####################################################################################################################################################################    
    def makeTk_DiMuonPairs(rdf):
    
        #this is to defne DiMuon pairs for different properties
        rdf = rdf.Define(f"TkDiMu_hwIsoSum", f"""
            ROOT::RVecI ret; 
            for (const auto & pair : TkDiMu_indices) 
                ret.emplace_back(0.03125*(std::max(TkMu_hwIsoSum[pair.first],TkMu_hwIsoSum[pair.second])));
            return ret;""")

        rdf = rdf.Define(f"TkDiMu_hwIsoSumAp", f"""
            ROOT::RVecI ret; 
            for (const auto & pair : TkDiMu_indices) 
                ret.emplace_back(0.03125*(std::max(TkMu_hwIsoSumAp[pair.first],TkMu_hwIsoSumAp[pair.second])));
            return ret;""")

        rdf = rdf.Define(f"TkDiMu_hwQual", f"""
            ROOT::RVecI ret; 
            for (const auto & pair : TkDiMu_indices) 
                ret.emplace_back(std::min(TkMu_hwQual[pair.first],TkMu_hwQual[pair.second]));
            return ret;""")

        return rdf;
    rdf_Tkdimu_one = makeDimuonPairs("Tk",rdf) 
    rdf_Tkdimu = makeTk_DiMuonPairs(rdf_Tkdimu_one)
    
    #h_TkDiMu_mass = rdf_Tkdimu.Histo1D(("","",50,0,50),"TkDiMu_mass")
    
    hNTkPairs = rdf_Tkdimu.Histo1D(("nTkDiMu","N(#mu#mu)_{Tk};Events",12,-2,10), "nTkDiMu")
    h_TkDiMu_mass = rdf_Tkdimu.Histo1D(("TkMass","m(#mu#mu)_{Tk};m (GeV);Pairs",50,0,50), "TkDiMu_mass")
    hTkPairCharge = rdf_Tkdimu.Histo1D(("TkCharge","q(#mu#mu)_{Tk};q(#mu_{1}) + q(#mu_{2});Pairs",8,-4,4), "TkDiMu_charge")
    hTkPair_Dz = rdf_Tkdimu.Histo1D(("TkPair_dz","dz(#mu#mu)_{Tk};z(#mu_{1}) - z(#mu_{2}cm",100,-10,10),"TkDiMu_dz")
    hTkPair_minPt = rdf_Tkdimu.Histo1D(("TkPair_minPt","min p_{T}(#mu#mu)_{Tk};pT GeV",100,0,10),"TkMu_minPT")
    hTkPair_Dz = rdf_Tkdimu.Histo1D(("TkPair_dz","dz(#mu#mu)_{Gen};z(#mu_{1}) - z(#mu_{2})cm",50,-10,10),"TkDiMu_dz")
    hTkPair_minPt = rdf_Tkdimu.Histo1D(("TkPair_minPt","min p_{T}(#mu#mu)_{Gen};pT GeV",50,0,10),"TkMu_minPT")
    hTkPair_hwIsoSum = rdf_Tkdimu.Histo1D(("Tk_IsoSum","Iso Sum(#mu#mu)_{Tk};;Pairs",20,0,20),"TkDiMu_hwIsoSum")
    hTkPair_hwIsoSumAp = rdf_Tkdimu.Histo1D(("Tk_IsoSumAp","Iso Sum Ap(#mu#mu)_{Tk};;Pairs",20,0,20),"TkDiMu_hwIsoSumAp")
    hTkPair_hwQual = rdf_Tkdimu.Histo1D(("Tk_Qual","Qual(#mu#mu)_{Tk};;Pairs",50,0,200),"TkDiMu_hwQual")
 
    entries_Tk_pair = rdf_Tkdimu.Count().GetValue()
    

    
####################################################################################################################################################################
    
    #hIdxTkPairMass = rdf_idxSel.Histo1D(("Tk_Mass_Idx","Idx m(#mu#mu)_{Tk};m (GeV);Pairs",50,0,50), "TkDiMu_IdxMass")

    def select_ind(name, rdf):
    
        #Let us impose a selection criteria here
        #rdf = rdf.Define(f"{name}DiMu_selind",f"{name}DiMu_indices")
        rdf = rdf.Define(f"{name}DiMu_good", f"({name}DiMu_charge<0.1 && {name}DiMu_charge>-0.1) &&({name}DiMu_dz<1 && ({name}DiMu_dz>-1))&&({name}Mu_minPT > 0.25*{name}DiMu_mass)&&({name}DiMu_eta <= 2) && (TkDiMu_hwQual > 75)&& (TkDiMu_hwIsoSumAp<=3) && (TkDiMu_hwIsoSum<=3)")                    
        rdf = rdf.Define(f"Good{name}DiMu_indices", f"{name}DiMu_indices[{name}DiMu_good]")
        rdf = rdf.Filter(f"Sum({name}DiMu_good)  >= 1")
        rdf = rdf.Define(f"n{name}DiMu_sel",f"Good{name}DiMu_indices.size()")
        rdf = rdf.Define(f"{name}DiMu_selMass",f"{name}DiMu_mass[{name}DiMu_good]")
        rdf = rdf.Define(f"{name}DiMu_selCharge",f"{name}DiMu_charge[{name}DiMu_good]")
        rdf = rdf.Define(f"{name}DiMu_seldz",f"{name}DiMu_dz[{name}DiMu_good]")
        rdf = rdf.Define(f"{name}DiMu_selMinPt",f"{name}Mu_minPT[{name}DiMu_good]")
        rdf = rdf.Define(f"{name}DiMu_selEta",f"{name}DiMu_eta[{name}DiMu_good]")
        rdf = rdf.Define(f"TkDiMu_selhwQual",f"TkDiMu_hwQual[{name}DiMu_good]")
        rdf = rdf.Define(f"TkDiMu_selhwIsoSum",f"TkDiMu_hwIsoSum[{name}DiMu_good]")
        rdf = rdf.Define(f"TkDiMu_selhwIsoSumAp",f"TkDiMu_hwIsoSumAp[{name}DiMu_good]")
#         rdf = rdf.Define(f"GoodTkDiMu_minIdx",f"TkDiMu_minIdx[TkDiMu_Idx_good]")
#         rdf = rdf.Define(f"TkDiMu_IdxselEta",f"TkDiMu_IdxEta[TkDiMu_Idx_good]")


        #rdf = rdf.Define(f"GoodTkDiMu_minIdx_{name}",f"TkDiMu_minIdx[{name}DiMu_good]")
        return rdf;

    rdf_Tk_selFeat = select_ind("Tk",rdf_Tkdimu)
    #rdf_selFeat = select_ind("Gen", rdf_Tk_selFeat)
        
    hNGoodIdxTkPairs = rdf_Tk_selFeat.Histo1D(("good nTk_DiMu_sel","Good N(#mu#mu)_{Tk};Events",10,0,10), "nTkDiMu_sel")
    hGoodIdxTkPairMass = rdf_Tk_selFeat.Histo1D(("Good Tk_Mass","Good m(#mu#mu)_{Tk};m (GeV);Pairs",50,0,50), "TkDiMu_selMass")
    hGoodIdxTkPairCharge = rdf_Tk_selFeat.Histo1D(("Good Tk_Charge","Good q(#mu#mu)_{Tk};q(#mu_{1}) + q(#mu_{2});Pairs",8,-4,4), "TkDiMu_selCharge")
    hGoodIdxTkPairDz = rdf_Tk_selFeat.Histo1D(("Good Tk dz","Good dz(#mu#mu)_{TK};z(#mu_{1}) - z(#mu_{2});Pairs",50,-10,10), "TkDiMu_seldz")
    hGoodIdxTkPairMinPT = rdf_Tk_selFeat.Histo1D(("Good Tk minPt","Good min pT(#mu#mu)_{Tk};pT(GeV));Pairs",50,0,20), "TkDiMu_selMinPt")
    hGoodIdxTkPairhwQual = rdf_Tk_selFeat.Histo1D(("Good Tk hw Qual","Good hw Qual(#mu#mu)_{Tk};);Pairs",50,0,200), "TkDiMu_selhwQual")
    hGoodIdxTkPairhwIsoSum = rdf_Tk_selFeat.Histo1D(("Good Tk hw Iso Sum","Good hw Iso Sum(#mu#mu)_{Tk};);Pairs",20,0,20), "TkDiMu_selhwIsoSum")
    hGoodIdxTkPairhwIsoSumAp = rdf_Tk_selFeat.Histo1D(("Good Tk hw Iso Sum Ap","Good hw Iso Sum Ap(#mu#mu)_{Tk};);Pairs",20,0,20), "TkDiMu_selhwIsoSumAp")
    hGoodIdxTkPairEta = rdf_Tk_selFeat.Histo1D(("Good TkDiMu Eta","Good TkDiMu lEta(#mu#mu)_{Tk};);Pairs",20,0,20), "TkDiMu_selEta")

       


    
    return dict(entries_no_pair = entries_no_pair, hNGoodIdxTkPairs = hNGoodIdxTkPairs, hGoodIdxTkPairMass = hGoodIdxTkPairMass, hGoodIdxTkPairCharge = hGoodIdxTkPairCharge, hGoodIdxTkPairDz = hGoodIdxTkPairDz, hGoodIdxTkPairMinPT = hGoodIdxTkPairMinPT, hGoodIdxTkPairhwQual = hGoodIdxTkPairhwQual, hGoodIdxTkPairhwIsoSum = hGoodIdxTkPairhwIsoSum, hGoodIdxTkPairhwIsoSumAp = hGoodIdxTkPairhwIsoSumAp, hGoodIdxTkPairEta = hGoodIdxTkPairEta);






In [6]:
# M = 15
# data_bg = Efficiency_plotter("l1MuNano_DYToLL_M-10To50_PU200.125X_v0.2.root",M)
# data_bg_low = Efficiency_plotter("l1MuNano_DYToLL_M-1To10_TuneCP5_14TeV-pythia8_PU200.125X_v0.2.root",M)
# data_bg_SM = Efficiency_SN("l1MuNano_SingleNeutrino_big_PU200.125X_v0.2.root",M)

# data_25 = Efficiency_plotter(f"l1MuNano_HAHM_ZdToMuMu_M{M}_PU200.125X_v0.2.root",M)


In [7]:
#8.0e6  (where you have instead  21.3e6 for the DY M-10To50). This is for DY
#cross_sec = [146.7, 92.96, 64.20, 21.53,7.459, (2e-2/5e-3)*((0.02**2)/3e-6), (2e-2/8e-3)*((0.02**2)/3e-6), (3e-2/2.5e-2)*((0.02**2)/3e-6) , (3e-2/3e-2)*((0.02**2)/3e-6), (3e-2/5e-2)*((0.02**2)/3e-6)]
# for j,i in enumerate(["10","12","15","25","40","2","3p5","4","5","7"]):

#(3e-6/(0.02**2))

In [8]:
# cross_sec = [(2e-2/5e-3)*((0.02**2)/3e-6), (3e-2/3e-2)*((0.02**2)/3e-6), 146.7, 64.20, 21.53]

# for k,M in enumerate(["2", "5", "10", "15", "25"]):
    
#     ##For the background
#     data_bg = Efficiency_plotter("l1MuNano_DYToLL_M-10To50_PU200.125X_v0.2.root",M)
#     data_bg_low = Efficiency_plotter("l1MuNano_DYToLL_M-1To10_TuneCP5_14TeV-pythia8_PU200.125X_v0.2.root",M)
#     data_bg_SM = Efficiency_SN("l1MuNano_SingleNeutrino_big_PU200.125X_v0.2.root",M)
    
#     #for the signal
#     data_25 = Efficiency_plotter(f"l1MuNano_HAHM_ZdToMuMu_M{M}_PU200.125X_v0.2.root",M)
    
#     c = ROOT.TCanvas("c","c", 2000,800)
# #     leg = ROOT.TLegend(0.7,0.7,0.9,0.9)
#     c.Divide(4,2)
#     for i,h in enumerate(["hNGoodIdxTkPairs", "hGoodIdxTkPairMass", "hGoodIdxTkPairCharge", "hGoodIdxTkPairDz", "hGoodIdxTkPairMinPT", "hGoodIdxTkPairhwQual","hGoodIdxTkPairhwIsoSum","hGoodIdxTkPairEta"]):
        
#         c.cd(i+1)
#         ##Scaling of the histograms
#         data_bg[h].GetPtr().Scale((21.3e6*400)/data_bg['entries_no_pair'])
#         data_bg_low[h].GetPtr().Scale((8.0e6*400)/data_bg_low['entries_no_pair'])
#         data_bg_SM[h].GetPtr().Scale((53.5e5*35e6)/data_bg_SM['entries_no_pair'])
#         data_25[h].GetPtr().Scale(cross_sec[k]/data_25['entries_no_pair'])

#         ##Adding backgrounds
#         data_bg[h].GetPtr().Add(data_bg_low[h].GetPtr())
#         data_bg[h] = Normal_h(data_bg[h].GetPtr(),data_25[h].GetPtr())
#         data_bg_SM[h] = Normal_h(data_bg_SM[h].GetPtr(),data_25[h].GetPtr())
#         data_25[h] = Normal_h(data_25[h].GetPtr(),data_25[h].GetPtr())
#         data_25[h].GetYaxis().SetRangeUser(0, 1.2*max(data_25[h].GetMaximum(), data_bg_SM[h].GetMaximum()))
#         data_25[h].SetTitle(f"{h} for M={M} signal")
#         #leg.AddEntry("data_25[h]","Signal","l")
#         data_25[h].Draw("hist")   

#         data_bg[h].SetLineColor(ROOT.kMagenta)
#         data_bg[h].SetTitle("Drell yan")
#         #leg.AddEntry("data_bg[h]","Drell Yan","l")
#         data_bg[h].Draw("hist SAME")

#         data_bg_SM[h].SetLineColor(ROOT.kGreen+1)
#         data_bg_SM[h].SetTitle("Single neutrino")
#         #leg.AddEntry("data_bg_SM[h]","single neutrino","l")
#         data_bg_SM[h].Draw("hist SAME")

#         #leg.Draw()



#     c.Draw()
#     c.SaveAs(f"mass_cut_All_para_signal_bg for M={M}.png")


In [27]:
# cross_sec = [(2e-2/5e-3)*((0.02**2)/3e-6), (3e-2/3e-2)*((0.02**2)/3e-6), 146.7, 64.20, 21.53]

# for k,M in enumerate(["2", "5", "10", "15", "25"]):
M=10
#     ##For the background
data_bg = Efficiency_plotter("l1MuNano_DYToLL_M-10To50_PU200.125X_v0.2.root",M)
data_bg_low = Efficiency_plotter("l1MuNano_DYToLL_M-1To10_TuneCP5_14TeV-pythia8_PU200.125X_v0.2.root",M)
data_bg_SM = Efficiency_SN("l1MuNano_SingleNeutrino_big_PU200.125X_v0.2.root",M)

#for the signal
data_25 = Efficiency_plotter(f"l1MuNano_HAHM_ZdToMuMu_M{M}_PU200.125X_v0.2.root",M)

c = ROOT.TCanvas("c","c", 1200,800)
#leg = ROOT.TLegend(0.7,0.7,0.9,0.9)
c.Divide(1,1)
#for i,h in enumerate(["hNGoodIdxTkPairs", "hGoodIdxTkPairMass", "hGoodIdxTkPairCharge", "hGoodIdxTkPairDz", "hGoodIdxTkPairMinPT", "hGoodIdxTkPairhwQual","hGoodIdxTkPairhwIsoSum","hGoodIdxTkPairEta"]):
for i,h in enumerate(["hGoodIdxTkPairMass"]):

    c.cd(i+1)
    ROOT.gPad.SetLogy(True)
    ##Scaling of the histograms
    data_bg[h].GetPtr().Scale((21.3e6*400)/data_bg['entries_no_pair'])
    data_bg_low[h].GetPtr().Scale((8.0e6*400)/data_bg_low['entries_no_pair'])
    data_bg_SM[h].GetPtr().Scale((53.5e5*35e6)/data_bg_SM['entries_no_pair'])
    data_25[h].GetPtr().Scale(146.7/data_25['entries_no_pair'])

    ##Adding backgrounds
    data_bg[h].GetPtr().Add(data_bg_low[h].GetPtr())
    #data_bg[h] = Normal_h(data_bg[h].GetPtr(),data_25[h].GetPtr())
    #data_bg_SM[h] = Normal_h(data_bg_SM[h].GetPtr(),data_25[h].GetPtr())
    #data_25[h] = Normal_h(data_25[h].GetPtr(),data_25[h].GetPtr())
    data_25[h].GetYaxis().SetRangeUser(0, 1.2*max(data_25[h].GetMaximum(), data_bg[h].GetMaximum(), data_bg_SM[h].GetMaximum()))
    data_25[h].SetTitle(f"{h} for M={M} signal")
    #leg.AddEntry("data_25[h]","Signal","l")
    #data_25[h].Draw("hist")   


    data_bg[h].SetLineColor(ROOT.kMagenta)
    data_bg[h].SetTitle("Drell yan")
    #leg.AddEntry("data_bg[h]","Drell Yan","l")
    data_bg[h].Draw("hist")
    data_bg[h].GetYaxis().SetRangeUser(100, 2*max(data_bg[h].GetMaximum(), data_bg_SM[h].GetMaximum()))

        
    
    

    data_bg_SM[h].SetLineColor(ROOT.kGreen+1)
    data_bg_SM[h].SetTitle("Single neutrino")
    #leg.AddEntry("data_bg_SM[h]","single neutrino","l")
    data_bg_SM[h].Draw("hist SAME")
    data_bg_SM[h].GetPtr().Fit("expo","W","",15,50)
    
    func = data_bg_SM[h].GetPtr().GetListOfFunctions().At(0)
    for m in 8., 10., 15, 25, 30:
        print("Histo at %3.0f GeV: %g" % (m, data_bg_SM[h].GetBinContent(data_bg_SM[h].GetXaxis().FindBin(m))))
        print("Fit   at %3.0f GeV: %g" % (m, func.Eval(m)))
    #leg.Draw()


c.SetLogy(True)
c.Draw()
c.SaveAs(f"mass_cut_All_para_signal_bg for M={M}.png")


Histo at   8 GeV: 3.868e+10
Fit   at   8 GeV: 1.20594e+10
Histo at  10 GeV: 1.42759e+10
Fit   at  10 GeV: 8.0003e+09
Histo at  15 GeV: 2.89377e+09
Fit   at  15 GeV: 2.86786e+09
Histo at  25 GeV: 0
Fit   at  25 GeV: 3.6852e+08
Histo at  30 GeV: 0
Fit   at  30 GeV: 1.32103e+08

****************************************
Minimizer is Minuit / Migrad
Chi2                      =  2.99289e+18
NDf                       =            6
Edm                       =      5385.46
NCalls                    =           97
Constant                  =      24.8546   +/-   1.64596     
Slope                     =    -0.205183   +/-   0.0949222   


Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Info in <TCanvas::Print>: png file mass_cut_All_para_signal_bg for M=10.png has been created


In [16]:
%jsroot on